# Open inTRACKtive WITHIN a Jupyter notebook

### Dependencies

In [1]:
%env ANYWIDGET_HMR=1
import pandas as pd

from pathlib import Path
from IPython.display import clear_output
from intracktive.notebook_widget import Widget
from intracktive.convert import dataframe_to_browser, make_track_ids_consecutive

env: ANYWIDGET_HMR=1


INFO:watchfiles.main:1 change detected
INFO:watchfiles.main:1 change detected
INFO:watchfiles.main:1 change detected
INFO:watchfiles.main:1 change detected
INFO:watchfiles.main:1 change detected


### Load data

In [2]:
# df = pd.read_csv("https://public.czbiohub.org/royerlab/zoo/C_elegans/tracks.csv")
df = pd.read_csv(
    "https://public.czbiohub.org/royerlab/ultrack/inTRACKtive/2D_multicolor_tracks_with_hex.csv"
)
df = make_track_ids_consecutive(df)
df

,track_id,t,y,x,parent_track_id,CellColor
0,1,0,8.0,473.0,-1,8655811
1,1,1,9.0,470.0,-1,7347372
2,1,2,9.0,469.0,-1,7018912
3,1,3,9.0,469.0,-1,7542682
4,1,4,9.0,468.0,-1,7738780
...,...,...,...,...,...,...
96252,1390,297,1435.0,18.0,-1,3038725
96253,1390,298,1435.0,24.0,-1,939273
96254,1390,299,1432.0,15.0,-1,3092751
96255,1391,298,1432.0,1897.0,-1,65638


### Open inTRACKtive widget

In [3]:
zarr_data_url = dataframe_to_browser(df, Path(), flag_open_browser=False)
clear_output()

widget = Widget(dataset_url=zarr_data_url)
widget

INFO:root:Serving /var/folders/0x/59zp_92x3dsgz9ms3l24tbsm0000gp/T/tmpoo8f21vr at http://127.0.0.1:8000
INFO:root:Server running...


Widget(dataset_url='http://127.0.0.1:8000/zarr_bundle.zarr/')

In [4]:
widget.get_selected_tracks

[]

In [5]:
widget.select_tracks([0])

In [6]:
def find_lineage_with_most_divisions():
    pass


def find_longest_track():
    pass